In [1]:
##### from Evaluator import Evaluator

evaluator = Evaluator()
evaluator.evaluate()

NameError: name 'Evaluator' is not defined

In [2]:
from DBMSEvaluator import DBMSEvaluator
from MySQLList import MySQLList
from PostgresList import PostgresList
from Neo4jList import Neo4jList
from MySQLCol import MySQLCol
from PostgresCol import PostgresCol
from Neo4jCol import Neo4jCol

# print("Evaluate MySQL (all features and labels in columns)")
# mysql_col_evaluator = DBMSEvaluator(MySQLCol)
# mysql_col_evaluator.evaluate()

# print("Evaluate MySQL (all features and labels in lists)")
# mysql_list_evaluator = DBMSEvaluator(MySQLList)
# mysql_list_evaluator.evaluate()

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# print("Evaluate Postgres (all features and labels in lists)")
# postgres_list_evaluator = DBMSEvaluator(PostgresList)
# postgres_list_evaluator.evaluate()

In [3]:
# from DBMSEvaluator import DBMSEvaluator
# from MySQLList import MySQLList
# from PostgresList import PostgresList
# from Neo4jList import Neo4jList
# from MySQLCol import MySQLCol
# from PostgresCol import PostgresCol
# from Neo4jCol import Neo4jCol
# print("Evaluate Postgres (all features and labels in columns)")
# postgres_col_evaluator = DBMSEvaluator(PostgresCol)
# postgres_col_evaluator.evaluate()

In [1]:
from CRUD import CRUD_Evaluator
from DBMSEvaluator import DBMSEvaluator
from MySQLList import MySQLList
from PostgresList import PostgresList
from Neo4jList import Neo4jList
from MySQLCol import MySQLCol
from PostgresCol import PostgresCol
from Neo4jCol import Neo4jCol
import pandas as pd

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
num_nodes = 1000
num_edges = "5_edges" #"scale_free"
crud_evaluator = CRUD_Evaluator(Neo4jList, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_neo4j_list.csv")
create_time = crud_evaluator.create()

Create


In [3]:
read_time, test_time = crud_evaluator.read(1, assert_labels = True, assert_features = True)

Read


Reading subgraphs of order 1:   2%|▉                                                  | 18/1000 [00:05<05:18,  3.09it/s]


KeyboardInterrupt: 

In [4]:
delete_time = crud_evaluator.delete()

Delete


In [1]:
from Neo4jConnector import Neo4jConnector axis 0 with siz
connector = Neo4jConnector()
connector.connect()

In [8]:

    print(res.single()["value"])

TypeError: 'NoneType' object is not subscriptable

In [12]:
import os

with connector.driver.session() as session:
    res = session.run("""CALL dbms.listConfig('server.directories.import');""")
    print(os.path.isfile(res.single()["value"] + "/X_and_y_1000_nodes_scale_free_col.csv"))
    

True


In [30]:
connector.driver.execute_query("MATCH (n)-[r]->(s) RETURN COUNT(DISTINCT r);")

EagerResult(records=[<Record COUNT(DISTINCT r)=0>], summary=<neo4j._work.summary.ResultSummary object at 0x7f44318ab010>, keys=['COUNT(DISTINCT r)'])

In [31]:
connector.driver.execute_query("MATCH (n) RETURN COUNT(n);")

EagerResult(records=[<Record COUNT(n)=0>], summary=<neo4j._work.summary.ResultSummary object at 0x7f443189a080>, keys=['COUNT(n)'])

In [22]:
from Neo4jQuery import Neo4jQuery
neo_query = Neo4jQuery(f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")
neo_query.intialize_column_data(f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")
neo_query.initialize_all_queries_columns(3)

In [32]:
import numpy as np
np.random.seed(42)
seed_node_ids = np.random.choice(np.arange(neo_query.X_and_y.shape[0]), size = 1000, replace = False)

In [33]:
with connector.driver.session() as session:
    res = session.run(neo_query.read_subgraph_query_dict[1], seed_node_id = seed_node_ids[0])
    print(res.to_df())

                                               edges  \
0  [[41987, 987231], [568917, 987231], [960759, 9...   

                                      idCollection  \
0  [41987, 492599, 568917, 755495, 960759, 987231]   

                           labels  \
0  [[0], [0], [0], [0], [0], [0]]   

                                            features  
0  [[0.8048236, 0.448808, 0.37314403, 0.99551404,...  


In [35]:
from InMemorySubgraphReader import InMemSubGraphReader
inmem_subgraph_reader = InMemSubGraphReader(f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")

In [38]:
inmem_subgraph_reader.get_subgraph_from_in_mem_graph(seed_node_ids[0], 1).features

array([], shape=(0, 100), dtype=float64)

In [40]:
inmem_subgraph_reader.edge_index.shape

(2, 2017119)

In [27]:
read_times, read_times_mem = dict(), dict()
for hops in range(3, 4, 1):
    read_time, test_time = crud_evaluator.read(hops, assert_labels = False, assert_features = False)
    read_times[hops] = read_time
    read_times_mem[hops] = test_time
update_node_time = crud_evaluator.update_nodes()
update_edge_time = crud_evaluator.update_edges()
delete_time = crud_evaluator.delete()
new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
for hops in read_times:
    new_row_dict[f"read_{hops}"] = read_times[hops]
    new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
new_row = pd.DataFrame([new_row_dict])

Read


Reading subgraphs of order 3:  43%|█████████████████████▌                            | 432/1000 [29:05<38:15,  4.04s/it]


KeyboardInterrupt: 

In [5]:
# num_nodes = 1000000
# num_edges = "scale_free"
# crud_evaluator = CRUD_Evaluator(PostgresCol, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")
# create_time = crud_evaluator.create()
# read_times, read_times_mem = dict(), dict()
# for hops in range(3, 4, 1):
#     read_time, test_time = crud_evaluator.read(hops, assert_reads=False)
#     read_times[hops] = read_time
#     read_times_mem[hops] = test_time
# update_node_time = crud_evaluator.update_nodes()
# update_edge_time = crud_evaluator.update_edges()
# delete_time = crud_evaluator.delete()
# new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
# for hops in read_times:
#     new_row_dict[f"read_{hops}"] = read_times[hops]
#     new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
# new_row = pd.DataFrame([new_row_dict])

In [4]:
# num_nodes = 1000
# num_edges =  "5_edges" #"scale_free"
# crud_evaluator = CRUD_Evaluator(MySQLCol, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")
# create_time = crud_evaluator.create()
# read_times, read_times_mem = dict(), dict()
# for hops in range(1, 4, 1):
#     read_time, test_time = crud_evaluator.read(hops, assert_reads=True)
#     read_times[hops] = read_time
#     read_times_mem[hops] = test_time
# update_node_time = crud_evaluator.update_nodes()
# update_edge_time = crud_evaluator.update_edges()
# delete_time = crud_evaluator.delete()
# new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
# for hops in read_times:
#     new_row_dict[f"read_{hops}"] = read_times[hops]
#     new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
# new_row = pd.DataFrame([new_row_dict])

In [5]:
# num_nodes = 1000
# num_edges =  "5_edges" #"scale_free"
# crud_evaluator = CRUD_Evaluator(MySQLList, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_mysql_list.csv")
# create_time = crud_evaluator.create()
# read_times, read_times_mem = dict(), dict()
# for hops in range(1, 4, 1):
#     read_time, test_time = crud_evaluator.read(hops, assert_reads=True)
#     read_times[hops] = read_time
#     read_times_mem[hops] = test_time
# update_node_time = crud_evaluator.update_nodes()
# update_edge_time = crud_evaluator.update_edges()
# delete_time = crud_evaluator.delete()
# new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
# for hops in read_times:
#     new_row_dict[f"read_{hops}"] = read_times[hops]
#     new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
# new_row = pd.DataFrame([new_row_dict])

In [6]:
# num_nodes = 1000
# num_edges =  "5_edges" #"scale_free"
# crud_evaluator = CRUD_Evaluator(PostgresList, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_postgres_list.csv")
# create_time = crud_evaluator.create()
# read_times, read_times_mem = dict(), dict()
# for hops in range(1, 2, 1):
#     continue
#     read_time, test_time = crud_evaluator.read(hops, assert_reads=True)
#     read_times[hops] = read_time
#     read_times_mem[hops] = test_time
# update_node_time = crud_evaluator.update_nodes()
# update_edge_time = crud_evaluator.update_edges()
# delete_time = crud_evaluator.delete()
# new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
# for hops in read_times:
#     new_row_dict[f"read_{hops}"] = read_times[hops]
#     new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
# new_row = pd.DataFrame([new_row_dict])

In [2]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(f"bolt://localhost:{7688}", auth=("neo4j", "password"))

In [3]:
driver.execute_query("MATCH (n) RETURN COUNT(n);")

EagerResult(records=[<Record COUNT(n)=0>], summary=<neo4j._work.summary.ResultSummary object at 0x7fc59bcf9240>, keys=['COUNT(n)'])

In [14]:
driver.execute_query("MATCH (n) DETACH DELETE(n);")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7fcb9cc5a8f0>, keys=[])

In [4]:
num_nodes = 1000
num_edges =  "5_edges" #"scale_free"
crud_evaluator = CRUD_Evaluator(Neo4jList, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_neo4j_list.csv")
create_time = crud_evaluator.create()
read_times, read_times_mem = dict(), dict()
for hops in range(1, 2, 1):
    read_time, test_time = crud_evaluator.read(hops, assert_reads=True)
    read_times[hops] = read_time
    read_times_mem[hops] = test_time
update_node_time = crud_evaluator.update_nodes()
update_edge_time = crud_evaluator.update_edges()
delete_time = crud_evaluator.delete()
new_row_dict = {"name": f"{str(num_nodes)}_nodes_{num_edges}", "create": create_time, "update_nodes": update_node_time, "update_edges": update_edge_time, "delete": delete_time}
for hops in read_times:
    new_row_dict[f"read_{hops}"] = read_times[hops]
    new_row_dict[f"read_in_mem_{hops}"] = read_times_mem[hops]
new_row = pd.DataFrame([new_row_dict])

Create
Read


Reading subgraphs of order 1:   0%|          | 0/1000 [00:00<?, ?it/s]

Update nodes


Updating nodes:   0%|          | 0/1000 [00:00<?, ?it/s]

Update edges


Updating edges:   0%|          | 0/1000 [00:00<?, ?it/s]

Delete


In [13]:
driver.execute_query("MATCH (n) RETURN n.label LIMIT 1;")

EagerResult(records=[<Record n.label=None>], summary=<neo4j._work.summary.ResultSummary object at 0x7fcb9d4f5480>, keys=['n.label'])

In [8]:
delete_time = crud_evaluator.delete()

Delete


KeyboardInterrupt: 

In [ ]:
new_row

In [ ]:
# with conn.cursor() as c:
#     c.execute("SELECT * FROM nodes;")
#     print(c.fetchone())

In [ ]:
# import psycopg2
# from psycopg2 import sql
# conn = psycopg2.connect(
#                 dbname="postgres",  # Connect to default db to create new db
#                 user='postgres',
#                 password='password',
#                 host='localhost')
# conn.autocommit = True
# with conn.cursor() as cursor:
#     cursor.execute(sql.SQL("DROP DATABASE IF EXISTS {};").format(sql.Identifier("X_and_y_1000_nodes_scale_free_col")))
#     print(f"Database 'X_and_y_1000_nodes_scale_free_col' deleted successfully.")

In [ ]:
# with conn.cursor() as cursor:
#     cursor.execute("""SELECT table_name FROM information_schema.tables
#            WHERE table_schema = 'public'""")
#     print(cursor.fetchall())
#     for table in cursor.fetchall():
#         print(table)

In [ ]:
# num_nodes = 1000
# num_edges = "5_edges"
# crud_evaluator = CRUD_Evaluator(Neo4jCol, f"X_{str(num_nodes)}_nodes_{num_edges}.csv", f"y_{str(num_nodes)}_nodes_{num_edges}.csv", f"edge_index_{str(num_nodes)}_nodes_{num_edges}.csv", f"X_and_y_{str(num_nodes)}_nodes_{num_edges}_col.csv")

# crud_evaluator.delete()